# Feature Engineering

In this notebook, I will create my target variable (whether or not the song appears on the billboard hot 100's) by seeing if the song exists in either of two billboard hot 100 dataframes found on the internet. 

In [2]:
import pandas as pd
import numpy as np
import math
from difflib import SequenceMatcher as sm # For comparing similarity of lyrics

Loading in my newly cleaned dataframe as well as my billboard data

In [3]:
df = pd.read_csv('./clean_df',index_col=0)

In [4]:
bb = pd.read_csv('./songs_1970_2018_uniq.csv', index_col=0, engine='python', parse_dates=['date'])

In [5]:
billboard = pd.read_csv('./Billboard_Hot100_Singles_1955-2013.csv', parse_dates=['Date'], skiprows=38)

In [6]:
billboard.set_index('Date',inplace=True)

Examining all three of my dataframes

In [7]:
df.head()

,lyrics,year,title,artist
0,My servants began to forge what was to become...,1999-04-20,?,MF Doom
1,"""Things take a turn for the worse"" ""Send him b...",2009-03-24,Absolutely,MF Doom
2,One more beer And I'll take you all All of yo...,2002-01-01,All Outta Ale,MF Doom
3,"Yea, that's right It's not a Hardy Boy myster...",2009-03-24,Angelz,MF Doom
4,"Oh my my my I'm feeling high My moneys gone, ...",NaN,Apple Tree (MF Doom Special Blend),MF Doom


In [8]:
billboard.head()

,Year,Peak Chart Position,Artist - Source Database,Artist - Modified,Title - Source Database,Title - Modified,Duration,Original Record Label & Catalog No.,Suggested Filename,Concatenate,Eliminate Hyphens,Substitute Google Comma Char.,Eliminate Ampersand Phrases,Unnamed: 14,Unnamed: 15,Eliminate Parentheticals,Unnamed: 17,Search Formula,Search
Date,,,,,,,,,,,,,,,,,,,
2055-01-01,1955.0,8,"Cowboy Church Sunday School, The",Cowboy Church Sunday School,Open Up Your Heart (And Let The Sunshine In),Open Up Your Heart (And Let the Sunshine In),2:53,Decca 9-29367,Cowboy Church Sunday School--Open Up Your Hear...,%22Cowboy Church Sunday School%22+%22Open Up Y...,%22Cowboy Church Sunday School%22+%22Open Up Y...,%22Cowboy Church Sunday School%22+%22Open Up Y...,NaN,NaN,%22Cowboy Church Sunday School%22+%22Open Up Y...,57.0,81.0,https://www.google.com/search?q=%22Cowboy Chur...,https://www.google.com/search?q=%22Cowboy Chur...
2055-01-08,1955.0,1,"McGuire Sisters, The",McGuire Sisters,Sincerely,Sincerely,2:54,Coral 9-61323,McGuire Sisters--Sincerely (1955),%22McGuire Sisters%22+%22Sincerely%22,%22McGuire Sisters%22+%22Sincerely%22,%22McGuire Sisters%22+%22Sincerely%22,NaN,NaN,%22McGuire Sisters%22+%22Sincerely%22,NaN,NaN,https://www.google.com/search?q=%22McGuire Sis...,https://www.google.com/search?q=%22McGuire Sis...
2055-01-08,1955.0,8,David Carroll and His Orchestra,David Carroll & His Orchestra,Melody Of Love,Melody of Love,2:47,Mercury 70516,David Carroll & His Orchestra--Melody of Love ...,%22David Carroll & His Orchestra%22+%22Melody ...,%22David Carroll & His Orchestra%22+%22Melody ...,%22David Carroll & His Orchestra%22+%22Melody ...,18.0,33.0,%22David Carroll%22+%22Melody of Love%22,NaN,NaN,https://www.google.com/search?q=%22David Carro...,https://www.google.com/search?q=%22David Carro...
2055-01-08,1955.0,17,Sunny Gale,Sunny Gale,"Let Me Go, Lover!","Let Me Go, Lover",2:30,RCA Victor 5952,"Sunny Gale--Let Me Go, Lover (1955)","%22Sunny Gale%22+%22Let Me Go, Lover%22","%22Sunny Gale%22+%22Let Me Go, Lover%22",%22Sunny Gale%22+%22Let Me Go%2C Lover%22,NaN,NaN,%22Sunny Gale%22+%22Let Me Go%2C Lover%22,NaN,NaN,https://www.google.com/search?q=%22Sunny Gale%...,https://www.google.com/search?q=%22Sunny Gale%...
2055-01-15,1955.0,3,Four Aces Featuring Al Alberts,Four Aces,Melody Of Love,Melody of Love,2:45,Decca 29395,Four Aces--Melody of Love (1955),%22Four Aces%22+%22Melody of Love%22,%22Four Aces%22+%22Melody of Love%22,%22Four Aces%22+%22Melody of Love%22,NaN,NaN,%22Four Aces%22+%22Melody of Love%22,NaN,NaN,https://www.google.com/search?q=%22Four Aces%2...,https://www.google.com/search?q=%22Four Aces%2...


In [9]:
bb.head()

,title,artist,peakPos,lastPos,weeks,rank,change
date,,,,,,,
2018-06-21,4th Dimension,KIDS SEE GHOSTS Featuring Louis Prima,42,0,1,42,New
2018-06-21,All Girls Are The Same,Juice WRLD,41,49,5,41,8
2018-06-21,All Mine,Kanye West,11,11,2,31,-20
2018-06-21,Alone,Halsey Featuring Big Sean & Stefflon Don,70,73,9,74,-1
2018-06-21,Babe,Sugarland Featuring Taylor Swift,72,0,2,95,Re-Entry


Creating a column that has just the year as opposed to the full date

In [9]:
df.columns = ['lyrics', 'date', 'title', 'artist']

In [10]:
df['year'] = pd.to_datetime(df['date'], errors='coerce')

In [11]:
df['year'] = df.apply(lambda x: x['year'].year,1)

In [12]:
df.head()

,lyrics,date,title,artist,year
0,My servants began to forge what was to become...,1999-04-20,?,MF Doom,1999.0
1,"""Things take a turn for the worse"" ""Send him b...",2009-03-24,Absolutely,MF Doom,2009.0
2,One more beer And I'll take you all All of yo...,2002-01-01,All Outta Ale,MF Doom,2002.0
3,"Yea, that's right It's not a Hardy Boy myster...",2009-03-24,Angelz,MF Doom,2009.0
4,"Oh my my my I'm feeling high My moneys gone, ...",NaN,Apple Tree (MF Doom Special Blend),MF Doom,NaN


Creating a function that will compare two strings and return true if they are similar but not equal (testing if it's mispelled)

In [18]:
def stringsAreMispelled(s1, s2):    
    seqA = sm(None, s1, s2)
    seqB = sm(None, s2, s1)
    return (seqA.ratio() > 0.7 or seqB.ratio() > 0.7) and s1 != s2

Creating a function that will return true if two strings are similar or equal

In [19]:
def stringsAreSame(s1, s2):    
    seqA = sm(None, s1, s2)
    seqB = sm(None, s2, s1)
    return (seqA.ratio() > 0.7 or seqB.ratio() > 0.7) or s1 == s2

Creating a funciton that returns true if the two strings are equal

In [20]:
def stringsAreEqual(s1, s2):    
    seqA = sm(None, s1, s2)
    seqB = sm(None, s2, s1)
    return s1 == s2

Renaming my artists and title columns in my two billboard dataframes so I can merge on them 

In [21]:
billboard.rename(columns={'Artist - Modified': 'artist', 'Title - Modified': 'title_bboard'}, inplace=True)

In [22]:
bb.rename(columns={'title': 'title_bboard'}, inplace=True)

Appending the two billboard dataframes together, one ends at 2013 so I'll subset the one that contains 2014 and and after to only include those years

In [23]:
bb_subset = bb[bb.index >= '2014']

In [24]:
final_bb = billboard.append(bb_subset, ignore_index=False)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Testing my final billboard dataframe with A$ap Rocky

In [25]:
final_bb[final_bb['artist'] == 'A$AP Rocky']

,Artist - Source Database,Concatenate,Duration,Eliminate Ampersand Phrases,Eliminate Hyphens,Eliminate Parentheticals,Original Record Label & Catalog No.,Peak Chart Position,Search,Search Formula,...,Unnamed: 15,Unnamed: 17,Year,artist,change,lastPos,peakPos,rank,title_bboard,weeks
2012-11-17,"A$AP Rocky Featuring Drake, 2 Chainz & Kendric...",%22A$AP Rocky%22+%22F--kin Problems%22,3:57,NaN,%22A$AP Rocky%22+%22F--kin Problems%22,NaN,ASAP) Worldwide/Polo Grounds Digital,8,https://www.google.com/search?q=%22A$AP Rocky%...,https://www.google.com/search?q=%22A$AP Rocky%...,...,%22A$AP Rocky%22+%22F--kin Problems%22,NaN,2012.0,A$AP Rocky,NaN,NaN,NaN,NaN,F--kin Problems,NaN
2013-02-02,A$AP Rocky Featuring Skrillex & Birdy Nam Nam,%22A$AP Rocky%22+%22Wild for the Night%22,3:32,NaN,%22A$AP Rocky%22+%22Wild for the Night%22,NaN,A$AP Worldwide/Polo Grounds Digital,80,https://www.google.com/search?q=%22A$AP Rocky%...,https://www.google.com/search?q=%22A$AP Rocky%...,...,%22A$AP Rocky%22+%22Wild for the Night%22,NaN,2013.0,A$AP Rocky,NaN,NaN,NaN,NaN,Wild for the Night,NaN
2013-02-02,A$AP Rocky,%22A$AP Rocky%22+%22Long Live A$AP%22,4:49,NaN,%22A$AP Rocky%22+%22Long Live A$AP%22,NaN,A$AP Worldwide/Polo Grounds Digital,86,https://www.google.com/search?q=%22A$AP Rocky%...,https://www.google.com/search?q=%22A$AP Rocky%...,...,%22A$AP Rocky%22+%22Long Live A$AP%22,NaN,2013.0,A$AP Rocky,NaN,NaN,NaN,NaN,Long Live A$AP,NaN
2015-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A$AP Rocky,-14,84.0,62.0,98.0,L$D,3.0


Creating a dataframe that from both my cleaned dataframe and my bilboard dataframe. By merging on artist and keeping the titles of the songs from both dataframes, I am able to get a dataframe that looks like this (see merged.head below)

In [26]:
df2 = df

In [27]:
merged = df2.merge(final_bb, on='artist',how='inner')[['artist', 'title', 'title_bboard']]

Instatiating my target variable column with all zeros

In [35]:
merged['hot100'] = 0

In [38]:
merged.head()

,artist,title,title_bboard,hot100
0,A$AP Rocky,1Train,F--kin Problems,0
1,A$AP Rocky,1Train,Wild for the Night,0
2,A$AP Rocky,1Train,Long Live A$AP,0
3,A$AP Rocky,1Train,L$D,0
4,A$AP Rocky,☆☆☆☆☆ 5IVE $TAR$,F--kin Problems,0


The column title has the title of every song A\$AP Rocky did, and it repeats that song's title for however many songs A\$AP Rocky had on the billboard hot100's. He had 4 songs, so the title is repeated 4 times. This happens for every song, so for each row I can compare each artist's song to all of the artist's hits to assess if the song was a hit or not. This column (hot100) will end up being my target variable, so doing this accurately is crucial to my model's success.

Creating masks for titles that are the same (similar or equal), songs that have similar titles (or are 'mispelled'), and songs that have only equal titles found in my dataframe and the billboard charts.

In [28]:
%time misspelled = merged.apply(lambda row: stringsAreMispelled(row['title'], row['title_bboard']), 1)

CPU times: user 31.4 s, sys: 16 ms, total: 31.4 s
Wall time: 31.5 s


In [29]:
%time equal = merged.apply(lambda row: stringsAreEqual(row['title'], row['title_bboard']), 1)

CPU times: user 10.1 s, sys: 0 ns, total: 10.1 s
Wall time: 10.1 s


In [30]:
%time same = merged.apply(lambda row: stringsAreSame(row['title'], row['title_bboard']), 1)

CPU times: user 31.3 s, sys: 0 ns, total: 31.3 s
Wall time: 31.4 s


Filtering my merged dataframe with song titles that are found in both my dataframe and on the billboard charts.

In [65]:
hot100 = merged[same]

$893$ hits were found

In [66]:
len(hot100)

893

In [33]:
hot100.reset_index(inplace=True)

Creating a column that represents if the song was a hit or not. I instantiate it will all zeroes then I iterate through the rows and see if the title of the song was found on the billboard hot 100's.  

In [35]:
merged['hot100'] = 0

In [36]:
for i in merged[same].index:
    merged.loc[i,'hot100'] = 1

The number of hits found lines up with my hot100 dataframe

In [37]:
sum(merged['hot100'])

893

Checking my dataframe with its new column, hot100

In [38]:
merged.head()

,artist,title,title_bboard,hot100
0,A$AP Rocky,1Train,F--kin Problems,0
1,A$AP Rocky,1Train,Wild for the Night,0
2,A$AP Rocky,1Train,Long Live A$AP,0
3,A$AP Rocky,1Train,L$D,0
4,A$AP Rocky,☆☆☆☆☆ 5IVE $TAR$,F--kin Problems,0


In [70]:
merged[same].head()

,artist,title,title_bboard,hot100
140,A$AP Rocky,Fuckin' Problems,F--kin Problems,1
263,A$AP Rocky,L$D,L$D,1
274,A$AP Rocky,Long.Live.A$AP,Long Live A$AP,1
481,A$AP Rocky,Wild for the Night,Wild for the Night,1
661,2 Chainz,Birthday Song,Birthday Song,1


Now I will create my final dataframe, by merging only the hits onto the dataframe. The songs that are not hits will have NaN values in this column (which I'll label as hot100). I'll fill these nan's with zeros and they'll be my positive class

In [58]:
df_final = df.merge(merged.iloc[merged[one].index, :][['artist', 'title', 'hot100']].drop_duplicates(), how='left')

In [62]:
df_final.fillna(0, inplace=True)

In [63]:
sum(df_final['hot100'])

870.0

My positive class is 870 long. This unforutantely means that my classes are severely unbalanced, as my negative class has more than 15,000 observations. This will have to be dealt with later, but for now I must manually comb through my positive class where the titles are similar (and not equal) and see if my comparison function mislabeled a song as a positive class becuase its title was similar to that of a hit. If a song labeled as a hit is in fact not a hit, I'll have to relable it as zero. Instead of showing all of this, I will only show myself doing this one time so my process can be seen.

First I have to view a section of my dataframe to manually check for mispellings. I can see on the fourth row that the song 'Pull Up' was mistaken for the hit'Roll Up'. Since these are in fact not the same song, 'Pull Up' will have to be relabeled as a zero.

In [274]:
merged[misspelled][150:300]

,artist,title,title_bboard,hot100
191515,Jadakiss,You Make Me Wanna,U Make Me Wanna,1
192150,Coolio,"1, 2, 3, 4 (Sumpin' New)","1,2,3,4 (Sumptin' New)",1
192373,Coolio,It's All The Way Live,It's All the Way Live (Now),1
195606,Wiz Khalifa,Pull Up,Roll Up,1
196954,Wiz Khalifa,Word Hard Play Hard (Remix),"Work Hard, Play Hard",1
197273,RZA,Wu-Wear: The Garment Renaissance,Wu-Wear - the Garment Renaissance,1
197391,Juvenile,Back That Azz Up,Back That Thang Up,1
198011,Juvenile,My Life,In My Life,1
198518,Drake,0 to 100 / The Catch Up,0 To 100 / The Catch Up,1
199089,Drake,30 for 30 Freestyle,30 For 30 Freestyle,1


This song was an advertisement but got mistaken for a hit  so it must be dropped. 

In [3]:
df_final.drop(index = 10264, inplace=True) # advertisement

Giving the song 'Still Drake' a zero since it was not a hit.

In [283]:
df_final.loc[22081,'hot100'] = 0

After I checked all mispelled songs (about 300 in total), I ended up removing only about $38$ songs from my positive class

In [285]:
df_final['hot100'].sum()

832.0

Saving this dataframe as df_hot100

In [288]:
df_final.to_csv('./df_hot100.csv')